<div class="alert alert-block alert-danger">

# 2B: Predicting Births (COMPLETE)

*This notebook is intended for students who have completed:*
 
**All of Chapter 2**

</div>

<div class="alert alert-block alert-warning">

#### Summary of Notebook:

This notebook will delve into a data frame that contains the number of births per day in a typical large hospital, across the years 2000-2014. Student will explore things like the typical number of births per year, per month, and per day of the week. They will seek to identify patterns and model the data in a way that might help the hospital properly staff their maternity ward for certain days of the year.

#### Includes:

- Making meaningful predictions using the empty model
- Analyzing residuals and exploring how the mean balances residuals
- Explore making conditional predictions with the empty model

</div>

<div class="alert alert-block alert-success">

## Approximate time to complete Notebook: 45-60 Mins

</div>

In [ ]:
# This code will load the R packages we will use
suppressPackageStartupMessages({
    library(coursekata)
})

<img src="https://i.postimg.cc/jTgVzWLd/X2-Maternity-Ward.jpg" alt="Several newborn babies sleeping in a maternity ward" width = 70%>

## Staffing the Maternity Ward

Hospitals use complex staffing models to make sure they have the appropriate number of doctors, nurses, and support staff on call for mothers going into labor on any particular day. If hospitals were able to predict the exact number of births at their facilities on future days, they could improve efficiency and reduce instances of staff shortage. Can we use data to predict the number of births on a certain day, at a typical hospital in the US?

### Motivating Question: How many babies will be born tomorrow?

### The Dataset
##### Description
Shows the number of births per day at a typical, large hospital in the US every day from 2000-2014. Note: Due to privacy law, data from a specific hospital is unavailable. Instead, we took data on the total number of births in the US on these dates and applied a formula to project how many births occurred at a typical, large hospital on each of the dates. **The notebook can be completed as if this data is from a specific hospital.**

##### Variables
- `year`: Year (2000-2014)
- `month`: Month, where 1 = January and 12 = December
- `date_of_month`: Day number of month
- `day_of_week`: Day of week, where 1 = Monday and 7 = Sunday
- `births`: Number of births


##### Data Sources 
 - Full birth data (for total births across US) was compiled by Five Thirty Eight, originally from the Social Security Administration (https://github.com/fivethirtyeight/data/tree/master/births)
 - CourseKata applied a formula to the total births to project the amount of births at a typical, large hospital during each of the same dates. The result is the dataset used in this notebook.


<div class="alert alert-block alert-success">

### 1.0 - Approximate Time:  10-15 mins

</div>

### 1.0 - Exploring the `baby_days` Dataset

**1.1 -** Run the following codeblock to download the dataset. The dataset shows the number of births per day at a typical, large hosptial in the US.

In [ ]:
# Download the dataset
baby_days <- read.csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vQFPYUVf36moVxfg7Amdds86UlNTxo7ISj9h1LAhfac-09J3K9HiHPNUsTP8vy8VSa5npeqhBT8SY_a/pub?output=csv")
head(baby_days)

**1.2 -** What years are represented in this dataset? How many total days are represented?

In [ ]:
# Potential way to call on each individual year
# Or just check the variable description
tally(~ year, data = baby_days)


# Sample Response to get the number of rows
str(baby_days)

<div class="alert alert-block alert-warning">

<b>Sample Response:</b>
 
There are 15 years represented in the dataset (2000 - 2014). Each of these years has 365 data points, one for each day of the year. Leap years (e.g., 2008) have 366 days.

There are 5479 days represented (each row represents a day).
</div>

**1.3 -** (If applicable) Find the number of people born at this hospital on the day that **YOU** were born. Or, look up a date you are interested in between 2000-2014.

In [ ]:
# Try modifying the code below to match your birthday
filter(baby_days, year == 2002 & month == 8 & date_of_month == 18)

<div class="alert alert-block alert-warning">

<b>Sample Response:</b> 

On my birthday (August 18th, 2002), there were 9 births at this hospital.

</div>

<div class="alert alert-block alert-info">

<b> <font size="+1">Key Question</font></b>

<br>

**1.4 -** Run the code below to create three separate visualizations in which you show the number of births by:

 - `year`
 - `month`
 - `day_of_week`

Comment on any trends you notice.

 </div>

In [ ]:
# using scatter plots
gf_point(births ~ year, data = baby_days) 
gf_point(births ~ month, data = baby_days)
gf_point(births ~ day_of_week, data = baby_days)

# using jitter plots
gf_jitter(births ~ year, data = baby_days) 
gf_jitter(births ~ month, data = baby_days)
gf_jitter(births ~ day_of_week, data = baby_days)

# using boxplots
gf_boxplot(births ~ factor(year), data = baby_days) 
gf_boxplot(births ~ factor(month), data = baby_days)
gf_boxplot(births ~ factor(day_of_week), data = baby_days)


<div class="alert alert-block alert-warning">

<b>Sample Response:</b> 

If they use jitter plots and box plots, they might wonder why there is a clump of dots up top *and* a clump of dots on the bottom. Remind them that each of these dots comes from an individual day. Which *days* consistently have lower births? You might have them put in `color = ~day_of_week` into those scatter/jitter plots.

While births are pretty consistent across years and months, they seem to vary a fair amount by day of week. Specifically, the weekend days (Saturday - 6 and Sunday - 7) seem to have fewer births than the other days of the week. 

Ask students to think about why this might be? They might wonder if somehow people don't want to give births on weekends. Some might consider hospitals and doctors' schedules: doctors who perform specialzied operations - like C-Sections - are in the hospital more often on weekdays. 

<b> Instructor Note:</b> 

Make a big deal about variation in births by day of week. This will help students when they start refining their models, later in the notebook.

</div>

In [ ]:
# using scatter plots with day_of_week colored
gf_point(births ~ year, color = ~day_of_week, data = baby_days) 
gf_point(births ~ month, color = ~day_of_week, data = baby_days)

# using jitter plots
gf_jitter(births ~ year, color = ~day_of_week, data = baby_days) 
gf_jitter(births ~ month, color = ~day_of_week, data = baby_days)

<div class="alert alert-block alert-success">

### 2.0 - Approximate Time:  15-20 mins

</div>

### 2.0 - Making predictions with the empty model

**2.1 -** If you were asked to predict the number of births on a randomly selected day from this dataset, what number of births would you predict? 

What if we used the empty model to predict the number of births? What is good about that prediction?

In [ ]:
# average number of births per day

# Use mean function
mean(~ births, data = baby_days)

# Or use favstats
favstats(~ births, data = baby_days)

# Or use lm()
lm(births ~ NULL, data = baby_days)

<div class="alert alert-block alert-warning">

<b>Sample Response (responses will vary):</b> 

Students might have idiosyncratic predictions.

The empty model would predict the mean daily number of births ~ 19. This is because the mean balances the positive and negative prediction errors, so it presents a "middle" estimate of the number of births on any given day, one that is not too high nor too low.

<b> Instructor Note:</b> 

It's ok for responses to vary here. The key thing to emphasize is that we don't know *anything* about the data, other than it will be chosen at random from the dataset. Hopefully, with this prompt, many students arrive at using the mean to make their predictions. If they don't, we'll explore in the following exercises why they may have wanted to (or not wanted to) choose the mean.

</div>

**2.2 -** Create and store an empty model for predicting the number of births per day as the variable `empty_model`. Then, display the fitted values of your model in GLM notation.

Here is some markdown to get you started:

$Y_i = b_0 + e_i$

In [ ]:
# Empty model
empty_model <- lm(births~NULL, data = baby_days)
empty_model

<div class="alert alert-block alert-warning">

<b>Sample Responses:</b> 

- $Y_i = 19.05 + e_i$

- $births_i = 19.05 + e_i$

</div>

**2.3 -** The code block below draws a random day from the dataset and shows the number of births on that day. Imagine you made a prediction for the number of births on that day using your empty model. Spoiler alert -- your prediction will probably be wrong.

What kind of error would this prediction make? What is the practical consequence of making this type of error?

In [ ]:
##Number of births from a random day
set.seed(123) # set seed to have consistent results
random_day <- sample(baby_days, 1) # select random row from dataset 

random_day

In [ ]:
# They could calculate residual or just eyeball the amount

# Sample Response
28 - 19.05

# Sample Response
random_day$births - predict(empty_model)[1]

<div class="alert alert-block alert-warning">

<b>Sample Response:</b> 

We underestimated the number of births on September 28th, 2006 by about 9 births. One practical consequence could be that we schedule too few doctors, nurses, and staff at the hospital (due to our faulty prediction), leading to too few staff to handle the number of births in the hospital that day. An overstretched staff could be dangerous for patients, especially those needing attention due to complications.

</div>

**2.4 -** Find all the predictions and store them back into the data frame. Do the same thing with the residuals. 

(You might want to just call these new variables `predictions` and `residuals`.)

In [ ]:
# Sample Response
baby_days$predictions <- predict(empty_model)
baby_days$residuals <- resid(empty_model)

# The teacher may want to show the results like this to confirm
# BIRTHS = MODEL PREDICTION + RESIDUAL
# that leads back to the big idea: DATA = MODEL + ERROR
head(select(baby_days, births, predictions, residuals))

<div class="alert alert-block alert-info">

<b> <font size="+1">Key Question</font></b>

<br>

**2.5 -** If we sum up all the residuals, what would you expect to get? 

Now try it. Why do you get this result? Does this result justify using the mean to predict the number of births on a random day? Why or why not?

</div>

In [ ]:
print("Sum of all residuals:")
sum(baby_days$residuals)

<div class="alert alert-block alert-warning">

<b>Sample Response:</b> 

This number is basically zero (it's -0.0000000000022). We get zero because our model is the mean, and the mean perfectly balances the positive and negative errors in our dataset. 

- **Example response in favor of using the mean:** We get a "balanced" prediction, with systematically predicting too high or too low. This makes the mean a reasonable value for predicting the number of births on a random date.

- **Example response against using the mean:**  The consequences of overpredicting the number of births (paying too many staff shifts for a night) are less severe than the consequences for underpredicting the number of births (having too few staff to handle the number of births, which can be dangerous). So, an ideal model *wouldn't* balance the positive and negative residuals. Rather, it would obtain negative residuals much more often (i.e. overpredict much more often).

**Instructor Note:** 

Students may need some guidance on the scientific notation. In addition, some students may have trouble conceptualizing what it means to add up *all* the residuals. As a scaffold, ask them to imagine that our dataset had only 3 dates. For each day, we predict the mean and find the residual. These residuals add to zero. What does that mean? Would you expect some positive and some negative errors? What are the practical consequences for each type of error?

</div>

**2.6 -** Although the residuals may be balanced, they may also be large. Find the Sum of Squares (SS). Is it easy to interpret what this number means? Why or why not?

In [ ]:
# Sample Responses

sum(baby_days$residuals^2)

supernova(empty_model)

anova(empty_model)

<div class="alert alert-block alert-warning">

<b>Sample Response:</b> 

This number represents the sum of all the residuals squared. It's difficult to interpret: the units are now not births but, rather, births squared. In addition, the sum total depends not only on the amount of error per day, but also the amount of days in our dataset.

</div>

**2.7 -** A more interpretable measure of the "typical" error amount is the standard deviation. Calculate and interpret the standard deviation of births.

In [ ]:
# Sample Responses

sd(~ births, data = baby_days)
favstats(~ births, data = baby_days)


<div class="alert alert-block alert-warning">

<b>Sample Response:</b> 

Our model's predictions are typically off by about 7 births per day.

Note: Some students might consider getting the mean of the absolute value of the residuals. That's actually a great idea! But the standard deviation (by virtue of squaring first), tends to make very big residuals count much more than the mean of absolute residuals. That's why the standard deviation "penalizes" errors that are bigger. That might be reasonable to do because in a model, you are probably okay with small errors but you really want to penalize very large errors. Also there is a lot about the whole idea of squaring that is going to help us make a lot of statistics ideas fit together more elegantly! We'll talk about that more later as we parse SS and learn about PRE and other statistics.

Here is the code for that other statistic: `mean(~ abs(residuals), data = baby_days)`

Given that it doesn't penalize "large residuals" as much as standard deviation, you might consider asking students if this value will be smaller/larger than the standard deviation.
</div>

<div class="alert alert-block alert-success">

### 3.0 - Approximate Time:  20-25 mins

</div>

### 3.0 - Making Conditional Predictions

<div class="alert alert-block alert-info">

<b> <font size="+1">Key Question</font></b>

<br>


**3.1 -** Which dates had the largest residuals (either positive or negative)? What do you notice about these dates? Why is our model so wrong on these days?

</div>

In [ ]:
# Sample Response
arrange(baby_days, desc(abs(residuals)))

<div class="alert alert-block alert-warning">

**Sample Response:** 

Of the top 10 highest magnitude residuals, most are December 25th - i.e. Christmas. It could be that many hospital staff may not be working on Christmas, so people don't get procedures (like C-Sections) on this day. Students might also say things like "moms don't want their babies to have a Christmas birthday." 

Whatever the case might be, the low number of births on Christmas leads to overestimates by our model. 


**Note to Instructors:**

You may need to nudge students into the idea of arranging by the *absolute* residuals.
</div>

**3.2 -** Earlier, we noticed that births on the weekends tended to vary from births during the week. Let's explore this further. Make a new variable that says whether the day is a weekend or not.

In [ ]:
# Sample Response
baby_days$weekend1 <- baby_days$day_of_week > 5

# Sample Response
baby_days$weekend2 <- (baby_days$day_of_week == 6) | (baby_days$day_of_week == 7) 

# Sample Response
baby_days <- baby_days %>%
    mutate(weekend3 = day_of_week > 5)

# Sample Response
baby_days <- baby_days %>%
    mutate(weekend4 = ifelse(day_of_week > 5, 'weekend', 'weekday'))

head(baby_days)

**3.3 -** Now visualize and find the mean number of births per day, for weekends and for weekdays. What do you notice?

In [ ]:
# Sample Responses
favstats(births~weekend4, data = baby_days)

mean(births~weekend4, data = baby_days)

gf_jitter(births~weekend4, data = baby_days)

<div class="alert alert-block alert-warning">

<b>Sample Response:</b> 

Clearly, we have fewer births per day on weekends on average. This may be due to special procedures (like C-Sections) being scheduled on weekdays.

</div>

**3.4 -** Run the following code, which creates a new version of the dataset (`Dec_12_2014`) that includes all the dates up to (but not including) Saturday, December 13th, 2014.

In [ ]:
Dec_12_2014 <- baby_days[1:5460,]
head(Dec_12_2014)

<div class="alert alert-block alert-info">

<b> <font size="+1">Key Question</font></b>

<br>

#### Class Competition

**3.5 -** Using only the data in `Dec_12_2014`, predict the number of births on Saturday, December 13th, 2014. This is the date after the dataset ends. Note: Your prediction **must** be based on a mean of some subset (or full set) of data points in `Dec_12_2014`. Whoever has the lowest magnitude error wins the class competition! No cheating.

</div>

In [ ]:
## Sample Responses

# Example using full dataset
print('Mean births using full dataset')
mean(Dec_12_2014$births)

# Example using weekend dates
print('Mean births among weekend days in the dataset')
mean(filter(Dec_12_2014, weekend4 == "weekend")$births)

# Example using December 13th dates
print('Mean births among all Dec 13ths in the dataset')
mean(filter(Dec_12_2014, month == 12 & date_of_month == 13)$births)

<div class="alert alert-block alert-warning">

<b>Sample Responses:</b> 

- I used the mean number of births among all values in the new dataset - 19. I used this for my prediction because I wanted to have as many data values as possible informing my prediction.

- I used the mean number of births among weekend days - 9. I used this for my prediction because I knew that there was large variation between weekend and weekday birth numbers, and the date we're predicting falls on a Saturday.

- I used the mean number of births among all December 13th dates - 19. I used this for my prediction because we are finding the number of births on December 13th, so we may as well use all the historical Decemeber 13th data.

<b>Instructor Note:</b> 

As shown before, much of the variation in the number of births can be explained by weekend vs weekday. Since the new date falls on a Saturday, methods that generally find means of weekend dates will be most effective (they'll tend to predict lower numbers of births, and the actual number of births on December 13th, 2014 was quite low - 11).

</div>

<div class="alert alert-block alert-info">

<b> <font size="+1">Key Question</font></b>

<br>

**3.6 -** The instructor will now reveal the true number of births on Saturday, December 13th, 2014. Reflect on the error from your prediction. What direction was the error? How high was the error in magnitude? Why do you think this occurred?

</div>

<div class="alert alert-block alert-warning">

*Responses will vary.*


<b>Instructor Note:</b> 

The true number of births on December 13th, 2014 was 11. Presumably, students who took the weekend vs. weekday factor into account will perform the best. It's important to emphasize *why* they did the best - we see that a lot of variation in our data can be explained by weekenday vs weekend. So, using that variable to inform our prediction will lead to better predictions. This is more important than the number of data values included in our model. Emphasizing this point will lead well into future chapters, where we use different predictive variables to explain variation in the response variable.

</div>